# Select and Configure Theme

In [ ]:
#theme = 'grade3'   #classic
theme = 'oceans16' #dark mode
width = 1100 #px

# Scroll Down for Content

In [ ]:
import warnings #tech debt
import math
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
# styling
from IPython.display import HTML

#.css files were created using jupyter-themer and slightly modified:
# 1) text justification: added 2 lines to .rendered_html * + p block :
#      text-align: justify;
#      text-justify: inter-word;
# 2) fixing first paragraph of block:
#      created a copy of .rendered_html * + p block for a new
#        .rendered_html + p block (no asterisk!) and got rid of top margin
# 3) uniform line spacing of lines with and without inline latex:
#      increased lineheight from 130% to 200%
# 4) set bottom-margin of paragraphs to 1.5 em
#
# oceans16 specific:
# 1) added right-margin of 10 % for markdown text

#has to be last line of cell:
jt_style = open('./swap_invariants_'+theme+'.css').read()
jt_style = jt_style.replace('980px', str(width)+'px') #obviously hacky, but whatever
HTML('<style>'+jt_style+'</style>')

In [ ]:
#change style of plots
from jupyterthemes import jtplot
jtplot.style(theme=theme, context='notebook', figsize=(14,12))

#enable latex
fontsize = 22
plt.rc('text', usetex=True)
plt.rc('font', size=fontsize, family='serif')
plt.rc('axes', labelsize='x-large', titlesize='x-large')
plt.rc('xtick', labelsize=fontsize)
plt.rc('ytick', labelsize=fontsize)
plt.rc('legend', fontsize=fontsize)

In [ ]:
eps = 0.00001
to_kappa = lambda lmbda: lmbda/(1-lmbda)

def I_sum_x_j(x_arr, D):
    return D - sum(x_arr)

def I_prod_x_j(x_arr, D):
    n = len(x_arr)+1
    return (D/n)**n / np.prod(np.maximum(x_arr, eps)) #np.minimum to prevent division by 0

def I_chi_x_j(x_arr, chi, D):
    n = len(x_arr)+1
    return (chi*(D - sum(x_arr)) + (D/n)**n)/(chi + np.prod(x_arr))

# haven't worked out x_j form for I_theta, but (r, phi) representation is good enough for plotting
def I_theta(theta, D, resolution):
    n = 2
    bound = np.pi/2-eps
    #use sin(linspace) to get higher resolution near 0 and pi/2 (where most of the action of I_theta is)
    phi = np.pi/4 * (1 + np.sin(np.linspace(-bound, bound, resolution)))
    r = theta*(D/(np.sin(phi)+np.cos(phi))) + (1-theta)*np.sqrt(2*D**n/(n**n * np.sin(2*phi)))
    return [r*np.cos(phi), r*np.sin(phi)] #return x and y array for plotting

def I_C_x_j_iterations(x_arr, A, D, rel_error = 0.001, max_iterations = 100):
    n = len(x_arr) + 1
    x_arr = np.maximum(x_arr, eps)
    S = sum(x_arr)
    P = D/A * D/n
    for x_i in x_arr:
        P *= D/n/x_i
    
    x_j_approx = D #initial guess = average token balance
    for i in range(max_iterations):
        x_j_next = (x_j_approx ** 2 + P)/(2*x_j_approx + S + D/A - D)
        if 1 - rel_error < x_j_next/x_j_approx < 1 + rel_error:
            return (x_j_next, i)
        x_j_approx = x_j_next
    
    return (x_j_approx, -1)

In [ ]:
#n tokens - 2d visualization (trading x_i for x_j)
#mode in ['basics', 'theta', 'stableswap']
def visualize_invariants_2d(n, D, #basic parameters
                            zoom, equal_aspect, fixed_axis, resolution, #plot parameters
                            mode, lmbda, chi, with_theta, theta, A, rel_error): #mode parameters
    
    if mode != 'stableswap':
        assert(n==2)
    
    if mode == 'basics':
        kappa = to_kappa(lmbda)
        chi = kappa * D**(n-1)
    
    if theta == 0:
        theta = lmbda
        
    if A == 0:
        A = chi
    
    x_arr = [D/n] * (n-1)
    x_arr[0] = 0
    
    stop_y = fixed_axis if fixed_axis > 0 else I_chi_x_j(x_arr, chi, D)
    stop_x = D/n if zoom else stop_y
    
    #higher resolution closer to 0
    x = stop_x * np.linspace(0, 1, resolution)**2
    
    sum_x_j = np.zeros(resolution)
    chi_x_j = np.zeros(resolution)
    prod_x_j = np.zeros(resolution)
    if mode == 'basics':
        kappa_x_j = np.zeros(resolution)
    if mode == 'stableswap':
        C_x_j = np.zeros(resolution)
        C_iterations = np.zeros(resolution)
    
    for (k, x_i) in enumerate(x):
        x_arr[0] = x_i
        sum_x_j[k] = I_sum_x_j(x_arr, D)
        chi_x_j[k] = I_chi_x_j(x_arr, chi, D)
        prod_x_j[k] = I_prod_x_j(x_arr, D)
        if mode == 'basics':
            kappa_x_j[k] = I_chi_x_j(x_arr, kappa, D)
        if mode == 'stableswap':
            C_x_j[k], C_iterations[k] = I_C_x_j_iterations(x_arr, A, D, rel_error)
    
    fig, ax = plt.subplots()
    ax.plot(x, sum_x_j, label=r'$I_\Sigma$')
    ax.plot(x, chi_x_j, label=r'$I_\chi$')
    ax.plot(x, prod_x_j, label=r'$I_\Pi$')
    
    if mode == 'basics':
        ax.plot(x, kappa_x_j, label=r'$I_\kappa$')
    
    if mode == 'stableswap':
        ax.plot(x, C_x_j, label=r'$I_C$')
    
    if with_theta and n==2:
        theta_x_i, theta_x_j = I_theta(theta, D, resolution * (2 if zoom else 1))
        ax.plot(theta_x_i, theta_x_j, label=r'$I_\theta$')

    to_dec_str = lambda f,digits: str(int(f*10**digits)/10**digits)
    title = 'Invariants\n$n='+str(n) + ', D='+to_dec_str(D,1)
    if mode == 'basics':
        title += r', \lambda='+to_dec_str(lmbda,2) + r', \kappa='+to_dec_str(kappa,1)
    title += r', \chi='+to_dec_str(chi,1)
    if mode == 'stableswap':
        title += ', A='+to_dec_str(A,1)
    if with_theta and n==2:
        title += r', \theta='+to_dec_str(theta,1)
    title += '$'
    plt.title(title)
    plt.xlabel('$x_i$')
    plt.ylabel('$x_j$')
    plt.grid(True)
    ax.legend()
    ax.set_xlim(left = 0, right = stop_x)
    ax.set_ylim(bottom = D/2 if zoom else 0, top = stop_y)
    if equal_aspect:
        ax.set_aspect('equal', adjustable='box')
    plt.show()
    
    if mode == 'stableswap':
        fig, ax = plt.subplots()
        plt.title('Iterations until Convergences of $x_j$ for $I_C$')
        plt.xlabel('$x_i$')
        plt.ylabel('$Iterations$')
        ax.plot(x, C_iterations)
        plt.show()

def interact_2d(mode):
    _ = widgets.interact(visualize_invariants_2d,
         n            = widgets.IntSlider(min=2,max=4,step=1,value=2,continuous_update=False) if mode=='stableswap' else widgets.fixed(2),
         D            = widgets.FloatSlider(min=0.1,max=10,step=0.1,value=2,continuous_update=False),
         zoom         = widgets.Checkbox(value=False),
         equal_aspect = widgets.Checkbox(value=False),
         fixed_axis   = widgets.IntText(value=0,continuous_update=False),
         resolution   = widgets.IntText(value=50,continuous_update=False),
         mode         = widgets.fixed(mode),
         lmbda        = widgets.fixed(0) if mode!='basics' else widgets.FloatSlider(min=0.01,max=0.99,step=0.01,value=0.5,continuous_update=False,),
         chi          = widgets.FloatSlider(min=0.1,max=10,step=0.1,value=1,continuous_update=False) if mode!='basics' else widgets.fixed(0),
         with_theta   = widgets.Checkbox(value=True) if mode!='basics' else widgets.fixed(False),
         theta        = widgets.FloatSlider(min=0.01,max=0.99,step=0.01,value=0.5,continuous_update=False) if mode!='basics' else widgets.fixed(0),
         A            = widgets.FloatLogSlider(base=10,min=-3,max=6,step=0.5,value=1,continuous_update=False)  if mode=='stableswap' else widgets.fixed(0),
         rel_error    = widgets.FloatLogSlider(base=10,min=-10,max=-1,step=0.5,value=-3,continuous_update=False) if mode=='stableswap' else widgets.fixed(0),
    )

In [ ]:
# 3 tokens 3d visualization
def visualize_3d(D, chi, zoom, fixed_axis, up_down, left_right, resolution):
    n = 3
    
    stop_z = fixed_axis if fixed_axis > 0 else I_chi_x_j([0,0], chi, D)
    stop_x = D/n if zoom else stop_z
    
    #higher resolution closer to 0
    x_1 = np.outer(stop_x * np.linspace(0, 1, resolution)**2, np.ones(resolution))
    x_2 = x_1.copy().T
    sum_x_3 = np.outer(np.zeros(resolution), np.zeros(resolution))
    prod_x_3 = np.outer(np.zeros(resolution), np.zeros(resolution))
    chi_x_3 = np.outer(np.zeros(resolution), np.zeros(resolution))
    
    for i in range(resolution):
        for j in range(resolution):
            x_arr = [x_1[i][j], x_2[i][j]]
            sum_x_3[i][j] = I_sum_x_j(x_arr, D)
            if sum_x_3[i][j] < 0 or sum_x_3[i][j] > stop_z:
                sum_x_3[i][j] = np.nan
            prod_x_3[i][j] = I_prod_x_j(x_arr, D)
            if prod_x_3[i][j] < 0 or prod_x_3[i][j] > stop_z:
                prod_x_3[i][j] = np.nan
            chi_x_3[i][j] = I_chi_x_j(x_arr, chi, D)
            if chi_x_3[i][j] < 0 or chi_x_3[i][j] > stop_z:
                chi_x_3[i][j] = np.nan
    
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore') #technical debt of NaN cut-off handling
        fig = plt.figure()
        ax = plt.axes(projection ='3d')
        fig.set_size_inches(18,18)
        
        ax.plot_surface(x_1, x_2, sum_x_3, label=r'$I_\Sigma$')
        ax.plot_surface(x_1, x_2, chi_x_3, label=r'$I_\chi$')
        ax.plot_surface(x_1, x_2, prod_x_3, label=r'$I_\Pi$')
        
        ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$x_3$')
        ax.set_zlim(bottom = D/n if zoom else 0, top = stop_z)
        plt.title('Invariants\n'+r'$\chi='+str(int(chi*100)/100)+'$')
        
        ax.view_init(up_down, left_right)
        plt.show()

def interact_3d():
    widgets.interact(visualize_3d,
        D          = widgets.FloatSlider(min=0.1,max=10,step=0.1,value=3,continuous_update=False),
        chi        = widgets.FloatSlider(min=0.1,max=10,step=0.1,value=1,continuous_update=False),
        zoom       = widgets.Checkbox(value=False, description='zoom'),
        fixed_axis = widgets.IntText(value=2,continuous_update=False),
        up_down    = widgets.IntSlider(min=0,max=90,step=2,value=20,continuous_update=False),
        left_right = widgets.IntSlider(min=-90,max=90,step=2,value=0,continuous_update=False),
        resolution = widgets.IntText(value=50,continuous_update=False),
    )

In [ ]:
# amp decay
def amp_decay(n):
    resolution = 20
    x = np.linspace(1, resolution, resolution)
    #n different tokens, each with a balance of 1 in equilibrium, split into x pieces for nx pieces in total
    #first token has balance of 1 piece -> rho_1 = 1/x
    #nx-1 pieces remaining split evenly over remaining n-1 tokens hence
    #rho_i = (nx-1)/(n-1)/x
    rho_1 = 1/x
    rho_i = (n*x - 1)/(x*(n-1))
    decay = rho_1 * rho_i**(n-1)
    fig, ax = plt.subplots()
    
    ax.bar(x, decay, label='Amp Decay')
    plt.xlabel(r'$\frac{1}{\rho}$')
    plt.ylabel('Amp Factor Scaling')
    plt.xticks(np.arange(1,resolution+1,1))
    plt.yticks(np.arange(0,1.1,0.1))
    plt.title('StableSwap Amp Decay (Lower Bound)')
    plt.grid(True)
    plt.show()

def interact_decay():
    widgets.interact(amp_decay,\
        n = widgets.IntSlider(min=2,max=6,step=1,value=2,continuous_update=False),\
    )

In [ ]:
def calculate_Curve_D(x_arr, A, rel_error = 0.001, max_iterations = 100):
    n = len(x_arr)
    S = A*sum(x_arr)
    
    D_approx = sum(x_arr) #initial guess = D for pool in equilibrium
    for i in range(max_iterations):
        P = 1
        for x_i in x_arr:
            P *= D_approx/(n*x_i)
        
        D_next = (n*D_approx*P + S)/(A + (n+1)*P - 1)
        if 1 - rel_error < D_next/D_approx < 1 + rel_error:
            return (D_next, i)
        D_approx = D_next
    
    return (D_approx, -1)

def curve_D_2d(n, A, from_one, resolution, rel_error):
    x = np.linspace(0+eps,1-eps, resolution)
    D_ss = np.zeros(resolution)
    D_ss_nn = np.zeros(resolution)
    D_iterations = np.zeros(resolution)
    D_cp = np.zeros(resolution)
    x_arr = np.ones(n)/n
    for (i,x_i) in enumerate(x):
        if from_one:
            x_arr[0] = x_i/n
            x_arr[1] = (2-x_i)/n
        else:
            x_arr[0] = x_i/n
            for j in range(1,n):
                x_arr[j] = 1/n + (1 - x_i)/(n*(n-1))
        D_ss[i], D_iterations[i] = calculate_Curve_D(x_arr, A, rel_error)
        D_ss_nn[i], _ = calculate_Curve_D(x_arr, A*(n**n), rel_error)
        D_cp[i] = n * pow(math.prod(x_arr),1/n)
    
    fig, ax = plt.subplots()
    if from_one:
        plt.title(r'$D$ as a function of $A, (\frac{x}{n}, \frac{2-x}{n}, \frac{1}{n}, \ldots (n-2) \textnormal{ times} \ldots, \frac{1}{n})$')
    else:
        plt.title(r'$D$ as a function of $A, (\frac{x}{n}, \frac{1}{n}+\frac{\frac{x}{n}}{n-1}, \ldots (n-1) \textnormal{ times} \ldots, \frac{1}{n}+\frac{\frac{x}{n}}{n-1})$')
    plt.xlabel('$x$')
    plt.ylabel('$D$')
    ax.plot(x, D_ss_nn, label='StableSwap ($A=A n^n$)')
    ax.plot(x, D_ss, label='StableSwap ($A=A$)')
    ax.plot(x, D_cp, label='ConstantProduct')
    ax.legend()
    plt.show()
    
    fig, ax = plt.subplots()
    plt.title('Iterations until Convergence)')
    plt.xlabel('$x$')
    plt.ylabel('Iterations')
    ax.plot(x, D_iterations)
    plt.show()

def interact_D_2d():
    _ = widgets.interact(curve_D_2d,
         n          = widgets.IntSlider(min=2,max=10,step=1,value=2,continuous_update=False),
         A          = widgets.FloatLogSlider(base=10,min=-3,max=6,step=0.5,value=1,continuous_update=False),
         from_one   = widgets.Checkbox(value=True, description='take just from one (otherwise take from all equally)'),      
         resolution = widgets.IntText(value=200,continuous_update=False),
         rel_error  = widgets.FloatLogSlider(base=10,min=-10,max=-1,step=0.5,value=-3,continuous_update=False),
    )

In [ ]:
def visualize_D_3d(A, up_down, left_right, resolution, rel_error):
    x_1 = np.outer(np.linspace(eps, 1, resolution)**2, np.ones(resolution))
    x_2 = x_1.copy().T
    D = np.outer(np.zeros(resolution), np.zeros(resolution))
    
    for i in range(resolution):
        for j in range(resolution):
            D[i][j] = calculate_Curve_D([x_1[i][j], x_2[i][j]], A, rel_error)[0]
    
    fig = plt.figure()
    ax = plt.axes(projection ='3d')
    fig.set_size_inches(18,18)
    ax.plot_surface(x_1, x_2, D)
    ax.set(xlabel='$x_1$', ylabel='$x_2$', zlabel='$D$')
    plt.title('D as a function of $A, (x_1, x_2)$\n$A='+str(int(A*100)/100)+'$')
    ax.view_init(up_down, left_right)
    plt.show()

def interact_D_3d():
    widgets.interact(visualize_D_3d,
        A          = widgets.FloatLogSlider(base=10,min=-3,max=6,step=0.5,value=1,continuous_update=False),
        up_down    = widgets.IntSlider(min=0,max=90,step=2,value=20,continuous_update=False),
        left_right = widgets.IntSlider(min=-180,max=180,step=2,value=-124,continuous_update=False),
        resolution = widgets.IntText(value=50,continuous_update=False),
        rel_error  = widgets.FloatLogSlider(base=10,min=-10,max=-1,step=0.5,value=-3,continuous_update=False),
    )

## Basic Invariants

Assume a pool with $n$ different tokens with balances $x_i, i = 1, \ldots , n$ (we're mostly sticking to the notation of the original [curve whitepaper](https://curve.fi/files/stableswap-paper.pdf)).

First, consider the constant **sum** formula (where $D$ can be considered the depth of the pool):
$$ I_\Sigma := \left[ \sum_{i=1}^{n} x_i = D \right] $$

If we assume that there is an equal number of all tokens in the pool, i.e. $\forall i,j: x_i = x_j$ (we also call such a pool in equilibrium) then $x_i = \frac{D}{n}$ and hence if we were to apply the constant **product** formula instead, we'd get:
$$ I_\Pi := \left[ \prod_{i=1}^{n} x_i = \left( \frac{D}{n} \right)^n \right] $$

Now that we have two invariants $I_\Sigma$ and $I_\Pi$, any [convex combination](https://en.wikipedia.org/wiki/Convex_combination) of the two will yield a new invariant that lies between the two:

$$ I_\lambda := \lambda I_\Sigma + (1 - \lambda) I_\Pi \quad , \lambda \in (0,1) $$

Alternatively, we can divide both sides of $I_\lambda$ by $1-\lambda$ and define $\kappa = \frac{\lambda}{1-\lambda}$ to get an alternative form:

$$ I_\kappa := \kappa I_\Sigma + I_\Pi \quad , \kappa \in (0, \infty) $$

However, choosing $\kappa$ (or $\lambda$) independent of $D$ is suboptimal because it leads to our interpolation not being scale-free: If we consider a pool with $n$ tokens that's maximally out of balance, i.e. $\forall i > 1: x_i = 0$, we get:

$$ x_1 = \frac{1}{\kappa} \frac{D^n}{n^n} - D $$

which is not linear in $D$.

Thankfully, this problem is easily rectified by choosing $\chi := \kappa D^{n-1}$, giving rise to our scale-free invariant $I_\chi$:

$$ I_\chi := \chi I_\Sigma + I_\Pi $$

Plugging in, we get:

$$ I_\chi = \left[ \chi \sum_{i=1}^{n} x_i + \prod_{i=1}^{n} x_i = \chi D + \left( \frac{D}{n} \right)^n \right] $$

### Visualization

To get an idea what these invariants actually look like, let's plot them for a pool with $n = 2$ and $n = 3$ tokens respectively.

To that end, we need to express $x_j$ as a function of all other $x_i, i \neq j$, $D$, and $\chi$. With a bit of rearranging we get:
$$ x_j = \frac{\chi \left( D - \sum_{i \neq j} x_i \right) + \left( \frac{D}{n} \right)^n}{\chi + \prod_{i \neq j} x_i} $$

Try changing $D$ and see how $I_\kappa$ moves around while the other 3 invariants stay in place:

In [ ]:
interact_2d('basics')

In [ ]:
interact_3d()

So, as expected, we see that $I_\chi$ is sandwiched between $I_\Sigma$ and $I_\Pi$. One feature that might perhaps seem surprising is that unlike $I_\Pi$, $I_\chi$ does not diverge when any of the $x_i$ approach 0. The intuition that's leading one astray if one has that expectation is that a convex combination of the invariants is neither the cartesian nor the polar convex combination of the graphs representing them.

### Radial Distance Invariant

In fact, let's express $I_\Sigma$ and $I_\Pi$ in polar coordinates for the 2-dimensional case (i.e. $x_1 = r \cos \phi, x_2 = r \sin \phi$):

$$ \text{reminder:} \quad \sin \alpha + \cos \alpha = \sqrt{2} \sin \left( \alpha + \frac{\pi}{4} \right), \quad \sin \alpha \cos \alpha = \frac{\sin 2\alpha}{2} $$

\begin{align*}
I_\Sigma = \left[ r(\cos \phi + \sin \phi) = D \right] \quad & \text{ hence } \quad r = \frac{D}{\sin \phi + \cos \phi}, \quad \phi = \arcsin \left( \frac{D}{\sqrt{2}r} \right) - \frac{\pi}{4} \\
I_\Pi = \left[ r^2 \cos \phi \sin \phi = \left( \frac{D}{2} \right)^2 \right] \quad & \text{ hence } \quad r = \frac{D}{2} \sqrt{\frac{2}{\sin 2 \phi}}, \ \ \quad \phi = \frac{1}{2} \arcsin \left( \frac{D^2}{2 r^2} \right)
\end{align*}

Armed with these formulae, we can consider yet another invariant $I_\theta, \theta \in (0,1)$ that uses a convex combination of the radial distances of the corresponding points of $I_\Sigma$ and $I_\Pi$ for a given polar angle $\phi$ (i.e. shoot a ray from the origin at angle $\phi$, find the points of intersection with $I_\Sigma, I_\Pi$ and call them $p_\Sigma, p_\Pi$, and then find the correspoinding point $p_\theta = \theta p_\Sigma + (1-\theta) p_\Pi$ of $I_\theta$ (that also lies on the ray at a relative distance of $\theta$ between the two)). So in 2 dimensions we get:

$$ I_\theta^{(2d)} = \left[ r = \theta \left( \frac{D}{\sin \phi + \cos \phi} \right) + (1 - \theta) \left( \frac{D}{2} \sqrt{\frac{2}{\sin 2 \phi}} \right) \right] $$

Visualizing it along with the other variants:

In [ ]:
interact_2d('theta')

Now this approach could be extended to higher dimensions (i.e. more tokens) and would perhaps benefit from a formulation using projective geometry (which might allow us to transform all those pesky trigonometric functions into linear algebra instead).

For now however, we'll instead turn to the last step in the Curve whitepaper and reconstruct their stableswap invariant to see how it compares to the invariants we have so far.

## Curve Stableswap Invariant

The Curve whitepaper constructs $I_\chi$, observes, just as we did, that it intersects the axes (in their words: "However, it wouldn't support prices going far from the ideal price 1.0.") but then takes a different route to achieve the desired divergence along the axes than we have with our Radial Distance Invariant $I_\theta$ by instead making $\chi$ vary based on the token balances in the pool (i.e. turning $\chi$ into a function $\chi(\overrightarrow{x}), \overrightarrow{x} = (x_1, \ldots, x_n)$) which can take any positive value but tends towards 0 as the pool disequilibrates, hence pushing the overall invariant towards the constant product invariant $I_\Pi$:

$$ \chi(\overrightarrow{x}) = A \cdot \mathrm{Decay}(\overrightarrow{x}) $$

$A$ is the amplification factor which takes on the role of leverage (with larger $A$ pushing the pool invariant towards $I_\Sigma$ and hence reducing slippage on swaps, while $A=0$ takes us all the way back to $I_\Pi$).

To understand $\mathrm{Decay}$ we first consider the ratio $\rho_i = \frac{x_i}{D/n}$, which can be thought of as a normalized version of $x_i$ that specifies the token's relative abundance in the pool. A pool in equilibrium yields $\rho_i = 1$ for all its tokens, but as the pool disequilibrates, some token balances have to decrease while others increase and hence some $\rho_i$ will have to go down as other $\rho_j$ rise.

Now, if we were to use the constant product formula, $\rho_j$ would rise exactly enough so that the product of $\rho_i$ and $\rho_j$ remains constant. For example, in a pool with $x_1 = 2, x_2 = 2$ (and hence $\rho_1 = \rho_2 = 1$) one could buy one token of the first type for two tokens of the second type yielding $x_1 = 1, x_2 = 4$ (and hence $\rho_1 = \frac{1}{2}, \rho_2 = 2$). If we consider the same pool but consider the constant sum formula however, we could trade 1 token of the first kind for 1 token of the second kind yielding token balances of $x_1 = 1, x_2 = 3$ and consequently $\rho_1 = \frac{1}{2}, \rho_2 = \frac{3}{2}$ for a product of $\frac{3}{4} < 1$.

So, since $I_\chi$ always lies somewhere between $I_\Sigma$ and $I_\Pi$, $\prod_i \rho_i$ starts off at $1$ when the pool is in equilibrium, decreases monotonically as token balances drift out of balance, and tends towards $0$ as the relative abundance of even one token tends towards $0$. Therefore $\prod_i \rho_i$ is a valid choice for $\mathrm{Decay}(\overrightarrow{x})$ and in fact the one used by Curve:

$$ \chi(\overrightarrow{x}) = A \prod_{i=1}^n \frac{x_i}{D/n} = A \frac{\prod_i x_i}{(D/n)^n} $$

The following graph visualizes $\mathrm{Decay}$ for a pool with $n$ tokens where one token's $\rho_i = 1/x$ while the surplus is spread evenly among the other tokens. In fact, the following graph constitutes a lower bound because it actually uses a constant sum invariant, so with the actual stableswap invariant the decay would be even slower (i.e. longer bars):

In [ ]:
interact_decay()

Notice that there are plenty of other, readily available candidates for $Decay$ such as $\min_i(\rho_i)$ and that if $f$ is a valid choice for $Decay$, so is $f^s, s > 0$.

Plugging $\chi(\overrightarrow{x})$ into $I_\chi$ and diving by $\prod_i x_i$ we get:

$$ \frac{A n^n}{D} \sum_{i=1}^{n} x_i + 1 = A n^n + \underbrace{\frac{(D/n)^n}{\prod_{i=1}^n x_i}}_{=Decay^{-1}} $$

We observe that $A$ always comes with the factor $n^n$. Absorbing it into $A$ has two advantages: First, it reduces clutter, and secondly, and more importantly, it actually makes the overall shape of the invariant more comparable between different values of $n$ (otherwise, one would have to reduce $A$ as one increases $n$ - see depth comparison graph further down).

So with this final modification we arrive at the Curve/Stableswap invariant $I_C(A)$:

$$ I_C(A) := \left[ \frac{A}{D} \sum_{i=1}^{n} x_i + 1 = A + \frac{(D/n)^n}{\prod_{i=1}^n x_i} \right] $$

### Stableswap Numerics

Since $I_C$ has no explicit form for $x_j$ and $D$, we'll have to turn to iterative methods for finding them instead.

Our approach will be: Put $I_C$ into its [implicit form](https://en.wikipedia.org/wiki/Implicit_function) $F(z) = 0$ (where $z$ is either some $x_j$ or $D$) and then employ [Newton's method](https://en.wikipedia.org/wiki/Newton%27s_method) by iterating $G(z) = z - \frac{F(z)}{F'(z)}$ until the desired precision is reached.

#### Finding $x_j$

\begin{align*}
F(x_j)  & = A + \frac{(D/n)^n}{\prod_i x_i} - \frac{A}{D} \sum_i x_i - 1 \\
F'(x_j) & = - \frac{1}{x_j} \frac{(D/n)^n}{\prod_i x_i} - \frac{A}{D} \\
        & \stackrel{1.}{=} - \frac{1}{x_j} \left( \frac{A}{D} \sum_{i=1}^{n} x_i + 1 - A \right) - \frac{A}{D}
\end{align*}

1. use invariant to substitute for $\frac{(D/n)^n}{\prod_i x_i}$ <span style="color:red">(TODO: not clear why this [would be allowed/doesn't interfere with convergence] since technically it's only valid when $F(x_j) = 0$?)</span>

Now, using Newton's method:

\begin{align*}
G(x_j) & = x_j - \frac{F(x_j)}{F'(x_j)} \\
       & \stackrel{1.}{=} \frac{\left( \frac{A}{D} \sum_{i=1}^{n} x_i + 1 - A + \frac{A}{D} x_j \right) + \left( A + \frac{(D/n)^n}{\prod_i x_i} - \frac{A}{D} \sum_i x_i - 1 \right)}{\frac{1}{x_j} \left( \frac{A}{D} \sum_{i=1}^{n} x_i + 1 - A \right) + \frac{A}{D}} \\
       & \stackrel{2.}{=} \frac{x_j \left( \frac{A}{D} x_j + \frac{(D/n)^n}{\prod_i x_i} \right)}{\frac{A}{D} \sum_{i=1}^{n} x_i + 1 - A + \frac{A}{D} x_j} \\ 
       & \stackrel{3.}{=} \frac{x_j^2 + \frac{D}{A} \frac{(D/n)^n}{\prod_{i \neq j} x_i}}{\sum_{i=1}^{n} x_i + \frac{D}{A} - D + x_j} \\
\end{align*}

1. substitute, absorb minus sign into denominator, and convert to same denominator
2. simplify numerator and expand by $x_j$
3. expand by $\frac{D}{A}$

We verify with Curve's [3pool](https://github.com/curvefi/curve-contract/blob/b0bbf77f8f93c9c5f4e415bce9cd71f0cdee960e/contracts/pools/3pool/StableSwap3Pool.vy#L356) code and find that things check out:
\begin{align*}
  y & = x_j \\
  c & = \frac{D}{A n^n} \frac{(D/n)^n}{\prod_{i \neq j} x_i} \\
  b & = \sum_{i \neq j} x_i + \frac{D}{A n^n} \\
  y_{\text{next}} & = \frac{y^2 + c}{2 y + b - D} \\
    & = \frac{x_j^2 + \frac{D}{A n^n} \frac{(D/n)^n}{\prod_{i \neq j} x_i}}{2 x_j + \sum_{i \neq j} x_i + \frac{D}{A n^n} - D}
\end{align*}

Even though $\frac{D}{n}$ seems like the canonical choice for the initial guess of $x_j$, we actually have to use $D$. To see why, look at $G(x_j)$ and consider a pool that's arbitrarily far from equilibrium (i.e. $\forall i \neq j: x_i < \epsilon$). The numerator of $G(x_j)$ would grow arbitrarily large, while the denominator would tend towards $x_j + \frac{D}{A} - D$ and would hence be 0 or even negative if $x_j \leq D - \frac{D}{A}$. Since $A$ is an arbitrary parameter, $\frac{D}{A}$ can be arbitrarily small, therefore we have to choose $x_j = D$ to ensure a sensible value of the denominator at all times.

#### Finding $D$

We perform an essentially equivalent series of steps as we did for $x_j$ to find $D$.

\begin{align*}
F(D)  & = A D + D \frac{(D/n)^n}{\prod_{i=1}^n x_i} - A \sum_{i=1}^{n} x_i - D \\
F'(D) & = A + (n+1) \frac{(D/n)^n}{\prod_{i=1}^n x_i} - 1
\end{align*}

Again, using Newton's method:

\begin{align*}
G(D) & = D - \frac{F(D)}{F'(D)} \\
     & = \frac{\left( A D + (n+1) D \frac{(D/n)^n}{\prod_i x_i} - D \right) - \left( A D + D \frac{(D/n)^n}{\prod_i x_i} - A \sum_i x_i - D \right)}{A + (n+1) \frac{(D/n)^n}{\prod_i x_i} - 1} \\
     & = \frac{n D \frac{(D/n)^n}{\prod_i x_i} + A \sum_i x_i}{A + (n+1) \frac{(D/n)^n}{\prod_i x_i} - 1}
\end{align*}

Using $D_0 = \sum_i x_i$ for a pool in equilibrium, we can then iterate $D_{m+1} = G(D_m)$ until it converges to $G$'s fixed point $D$ (which again checks out with Curve's [3pool](https://github.com/curvefi/curve-contract/blob/b0bbf77f8f93c9c5f4e415bce9cd71f0cdee960e/contracts/pools/3pool/StableSwap3Pool.vy#L195) code).

Plotting $x_j$ as a function of $x_i$ (where all other $x_k, k \neq i, k \neq j$ are set to their equilibrium value $\frac{D}{n}$):

In [ ]:
interact_2d('stableswap')

Finally, let's plot $D$ as a function of the token balances $x_i$ and compare what the curves look like with and without absorbing $n^n$ in to $A$:

In [ ]:
interact_D_2d()

In [ ]:
interact_D_3d()